<a href="https://colab.research.google.com/github/Natdata/Dodatki---wsp-lni-u-ytkownicy/blob/main/Dodatki_wsp%C3%B3lni_widzowie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import requests
from pandas import json_normalize

# Function to fetch leaflet data from API
def fetch_leaflet_data(leaflet_id, start, end, headers):
    final_df = pd.DataFrame()
    data = {
        "event_name": "leaflet_enter",
        "event_properties": [{"name": "leaflet_id", "operator": "equals", "value": leaflet_id}],
        "from": start,
        "to": end
    }

    try:
        response = requests.post('https://eu1.api.clevertap.com/1/events.json', headers=headers, json=data)
        response.raise_for_status()  # Ensure we raise an exception for bad status codes
        response_json = response.json()
        req_id = response_json.get('cursor')

        while req_id:
            print(f"Fetching data for cursor: {req_id}")  # Logging the cursor to track progress
            params = (('cursor', req_id),)
            response1 = requests.get(f'https://eu1.api.clevertap.com/1/events.json?cursor={req_id}', headers=headers, params=params)
            response1.raise_for_status()  # Ensure we raise an exception for bad status codes
            response_data = response1.json()
            records = response_data.get('records', [])
            response_df = json_normalize(records)

            if 'ts' in response_df.columns:
                response_df = response_df[['ts', 'event_props.leaflet_id', 'profile.identity']]
                response_df.rename(columns={'event_props.leaflet_id': 'leaflet_id', 'profile.identity': 'user_id'}, inplace=True)
                response_df['ts'] = response_df['ts'].astype(str).apply(lambda x: x[:8])
                final_df = pd.concat([final_df, response_df], ignore_index=True)

            req_id = response_data.get('next_cursor')
            if not req_id:  # If no next cursor, break the loop
                break

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

    return final_df

# API headers
headers = {
    'X-CleverTap-Account-Id': 'R95-8KW-ZR5Z',
    'X-CleverTap-Passcode': '7dd63a92-b9c9-40c3-a004-330f9739e431',
    'Content-Type': 'application/json',
}

# Date range for the data
start = '2024-04-25'
end = '2024-04-30'

# Leaflet IDs
leaflet_ids = ['358984', '357688']  # 1ID - dodatek, 2ID Oryginał

# Convert dates to integers
start = int(start.replace('-', ''))
end = int(end.replace('-', ''))

# Fetch data for both leaflets
leaflet_data_1 = fetch_leaflet_data(leaflet_ids[0], start, end, headers)
leaflet_data_2 = fetch_leaflet_data(leaflet_ids[1], start, end, headers)

# Combine the data into a single DataFrame
conversion = pd.concat([leaflet_data_1, leaflet_data_2], ignore_index=True)

# Save the combined data to a CSV file
#conversion.to_csv('combined_leaflet_data.csv', index=False)

# Unique users for each leaflet
unique_users_leaflet_1 = set(leaflet_data_1['user_id'])
unique_users_leaflet_2 = set(leaflet_data_2['user_id'])

# Users only in leaflet 1
users_only_in_leaflet_1 = unique_users_leaflet_1 - unique_users_leaflet_2

# Users who entered leaflet 1 first, then leaflet 2
leaflet_data_1_sorted = leaflet_data_1.sort_values(by='ts')
leaflet_data_2_sorted = leaflet_data_2.sort_values(by='ts')

# Create a DataFrame with the first entry of each user in each leaflet
first_entry_leaflet_1 = leaflet_data_1_sorted.groupby('user_id').first().reset_index()
first_entry_leaflet_2 = leaflet_data_2_sorted.groupby('user_id').first().reset_index()

# Merge the DataFrames to find users who entered both leaflets
merged_entries = pd.merge(first_entry_leaflet_1, first_entry_leaflet_2, on='user_id', suffixes=('_leaflet_1', '_leaflet_2'))

# Find users who entered leaflet 1 first, then leaflet 2
unique_first_leaflet_1_then_leaflet_2 = merged_entries[merged_entries['ts_leaflet_1'] < merged_entries['ts_leaflet_2']]['user_id'].unique()

# Display results
print("Data has been saved to 'combined_leaflet_data.csv'.")

print("1. Liczba unikatowych użytkowników, którzy weszli w dodatek, ale nie weszli w oryginał:")
print(len(users_only_in_leaflet_1))

print("\n2. Liczba unikatowych użytkowników dodatku:")
print(len(unique_users_leaflet_1))

print("\n3. Liczba unikatowych użytkowników oryginału:")
print(len(unique_users_leaflet_2))

print("\n4. Liczba unikatowych użytkowników, którzy weszli najpierw w dodatek, a następnie w oryginał:")
print(len(unique_first_leaflet_1_then_leaflet_2))


Fetching data for cursor: ZyZjfgABAQdvb2t5Kz8NegUBCQNua2p%2BY29jeU4BBgVvbWt6b29mfgMCTQZmbS57K2oue04ATQQrai57endiNwMFAwVnaWp%2BbmguLm0BBgVvbWt6b29mfgMCTQVga2p8bmtqfgYFAAYraGN8K2oue04ATQQrai57K2p%2FZgJMAAFla2J4b29reU5VbgVga2p8bmtqfgYFAAYra2V6b21regoFBQFmaC55Zm0ue04ATQQrai57K2ouex8dAUhmb2B6Z2lqfgsCTVEIa2V6b21regoFBQFmaC56YGtqfAsBCQFjb2N5K2hjfE4ATQQrai57K2oue04AHBlnJmN%2BZWtieAoFCAYrPw16YGtqfAsBCQFjb2N5K2tlegoHCAVvb2Z%2BZmgueQMHTQQrai57K2oue04ATQR6d2I3Zm9gegIDCQFuaC4uCGtlegoHCAVvb2Z%2BZmguegUBCQNua2p%2BY29jeU4CAAMrai57K2oue04ATQQran9mZyZjfgABAQdvb2t5Kz8NegUBCQNua2p%2BY29jeU4BBgVvbWt6b29mfgMCTQZmbS57K2oue04ATQQrai57endiNwMFAwVnaWp%2BbmguLm0BBgVvbWt6b29mfgMCTQVga2p8bmtqfgYFAAYraGN8K2oue04ATQQrai57K2p%2FZgJMAAFla2J4b29reU5VbgVga2p8bmtqfgYFAAYra2V6b21regoFBQFmaC55Zm0ue04ATQQrai57K2ouex8dAUhmb2B6Z2lqfgsCTVEIa2V6b21regoFBQFmaC56YGtqfAsBCQFjb2N5K2hjfE4ATQQrai57K2oue04AHBlnJmN%2BZWtieAoFCAYrPw16YGtqfAsBCQFjb2N5K2tlegoHCAVvb2Z%2BZmgueQMHTQQrai57K2oue04ATQR6d2I3Zm9gegIDCQFuaC4uCGtlegoHCAVvb2Z%2BZmgu